In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from scipy.spatial.transform import Rotation as R  # rotation axis ??? left-hand / clockwise

from base import Camera, find_points
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

np.set_printoptions(suppress=True)

In [25]:
def _find(l, s):
    for elem in l:
        if elem['id'] == s or elem['id'] == f'{s}_0':
            values = elem['values']
            for i, value in enumerate(values):
                if value['labelName'] == 'cleaning_snuggle_henkel':
                    break
            else:
                return None
            return values[i]


folder_dir = './data/solo'
# folder_dir = './data/solo_basic'

ground_truth_coors = [
    np.array([0, 0, 15]),
    np.array([0, 3, 15]),
    np.array([3, 0, 18]),
    np.array([3, 3, 18]),
]

errs = []
for i in range(4):
    f_dir = f'{folder_dir}/sequence.{i}'
    with open(f'{f_dir}/step0.frame_data.json') as json_file:
        data = json.load(json_file)
    cameras = data['captures']

    rays = []
    for j, camera_info in enumerate(cameras):
        # if j == 0:
        #     continue
        # print('camera info', camera_info['position'], camera_info['rotation'])
        q_ = camera_info['rotation']

        camera = Camera(
            position=camera_info['position'],
            quaternion=np.array([q_[3], q_[0], q_[1], q_[2]]),
            resolution=camera_info['dimension'],
        )

        camera_coor = camera.world_to_camera([ground_truth_coors[i]])
        pixel_coor = camera.camera_to_pixel(camera_coor)
        # ray = camera.pixel_to_ray(pixel_coor)

        bbox_2d = _find(camera_info['annotations'], 'bounding box')
        center, size = bbox_2d['origin'], np.array([bbox/2 for bbox in bbox_2d['dimension']])
        ray = camera.pixel_to_ray([center+size])

        # fig, ax = plt.subplots()
        # ax.imshow(mpimg.imread(f'{folder_dir}/{camera_info["filename"]}'))
        # ax.add_patch(
        #     patches.Rectangle(
        #         center,
        #         # raster_coor[0],
        #         2*size[0], 2*size[1],
        #         linewidth=1, edgecolor='r', facecolor='none'
        #     )
        # )
        # plt.plot(
        #     raster_coor[0][0], raster_coor[0][1],
        #     "o", markersize=5, markeredgecolor="blue", markerfacecolor="green"
        # )
        # plt.axis('off')
        # plt.show()

        # print(camera_coor[0], pixel_coor[0], center+size)
        # print(ray[0])
        # print(f'{camera_info["id"]:<10} {raster_coor[0]} {center+size}')
        # break

        bbox_3d = _find(camera_info['annotations'], 'bounding box 3D') 
        # the bbox_3d['translation'] is the coordinate relative to the camera.
        # Therefore, we need to convert it to world coordinate
        obj_coor = np.array(bbox_3d['translation'])
        # print(np.array(bbox_3d['rotation']))
        
        # print(bbox_3d['rotation'], np.array(bbox_3d['translation']), ret)
        print(ray[0])
        rays.append(ray[0])

    # get prediction
    pred = find_points(*rays)

    # get ground truth location
    q_ = camera.quaternion
    ret = Camera.qm_2(
        np.array([q_[0], -q_[1], -q_[2], -q_[3]]),
        np.array(bbox_3d['translation'])
    ) + camera.position
    err = np.sqrt(((pred - ret)**2).sum())
    errs.append(err)
    print(f'{pred} {ret} {err:.4f}')
    
    print('-'*30)
    break
print(np.mean(errs))

Line(origin=array([0., 0., 0.]), direction=array([ 0.        , -0.00037588,  1.        ]))
Line(origin=array([15.,  0., 15.]), direction=array([-1.00000005, -0.00075176, -0.00000005]))
[  0.           0.00563819 -15.00000344] [-0.00000583 -0.01127638 14.9999992 ]
[-0.00000292 -0.00281909 -0.00000212] [-0.0000008  -0.         15.00000059] 15.0000
------------------------------
Line(origin=array([0., 0., 0.]), direction=array([0.        , 0.19996767, 1.        ]))
Line(origin=array([15.,  0., 15.]), direction=array([-1.00000005,  0.20034355, -0.00000005]))
[  0.          -3.12469722 -15.62601216] [-0.62601452  3.13057102 14.99999917]
[-0.31300726  0.0029369  -0.31300649] [-0.0000008   3.         15.00000095] 15.6067
------------------------------
Line(origin=array([0., 0., 0.]), direction=array([ 0.16689031, -0.00018794,  1.        ]))
Line(origin=array([15.,  0., 15.]), direction=array([-1.00000007, -0.0009397 ,  0.25127512]))
[ -3.10138952   0.00349256 -18.58340085] [ 2.20096511 -0.012

In [ ]:
camera_coor = [
    np.array([0, 0, 15]),
    np.array([3, 0, 15]),
    np.array([0, 3, 15]),
    np.array([3, 3, 15]),
]

focal = 20.78461
resolution = np.array([3840, 2160])
sensor = np.array([30, 30])
intrinsic = np.array([
    [focal*resolution[0]/sensor[0], 0, 0],
    [0, -focal*resolution[0]/sensor[0], 0],
    [resolution[0]/2, resolution[1]/2, 1],
])
ndc_coor = [np.matmul(coor, intrinsic) for coor in camera_coor]

pixel_coor = [np.floor((coor/coor[-1])[:-1]) for coor in ndc_coor]
for coor_a, coor_b in zip(pixel_coor, ndc_coor):
    print(coor_a, "|", coor_b/coor_b[-1])

In [ ]:
def camera_to_world(pixel_coor):
    homo_coor = [np.insert(coor, 2, 1) for coor in pixel_coor]

    coor = [np.matmul(coor, np.linalg.inv(intrinsic)) for coor in homo_coor]

    return coor

coors = camera_to_world(pixel_coor)
for coor in coors:
    print(coor)


In [ ]:
def qm(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    
    return np.array([
        -x1*x0 - y1*y0 - z1*z0 + w1*w0,
         x1*w0 - y1*z0 + z1*y0 + w1*x0,
         x1*z0 + y1*w0 - z1*x0 + w1*y0,
        -x1*y0 + y1*x0 + z1*w0 + w1*z0
    ])

q = np.array([0, 1, 0, 0])
abc = [
    np.array([0, 2, 1.5, 0]),
]
d = np.array([2, 2, 5])

_inv = np.array([1, -1, -1, -1])
print(
    qm(qm(q, abc[0]), q*_inv), qm(qm(q, abc[0]), q*_inv)[1:] - d
)

In [ ]:
abc = np.array([1, 2, 3])
np.insert(abc, 0, 0)

In [ ]:
np.zeros(3)

In [ ]:
v_a = np.array([1, 2, 3])
v_b = np.array([2, -1, 3])

cross = np.cross(v_a, v_b)
print(np.cross(v_a, v_b))
print(np.cross(v_b, v_a))

In [ ]:
def cross_product(u, v):
    return np.array([
        u[1]*v[2] - u[2]*v[1],
        u[2]*v[0] - u[0]*v[2],
        u[0]*v[1] - u[1]*v[0],
    ])
    
# print(cross_product(v_a, v_b))
print(cross_product(np.array([1, 0, 0]), np.array([0, 1, 0])))
print(cross_product(np.array([1, 0, 0]), np.array([0, 0, 1])))
print(cross_product(np.array([1, 0, 0]), np.array([0, 0, 1])))
print(cross_product(np.array([1, 0, 0]), np.array([0, 0, 1])))